In [2]:
import glassppy as glass
from ann_dataset import dataset_dict

In [3]:
topk = 10
dataset = dataset_dict['sift-128-euclidean']()
X_train = dataset.get_database()
X_test = dataset.get_queries()
Y = dataset.get_groundtruth(topk)
n, d = X_train.shape
nq, d = X_test.shape
metric = dataset.metric
print(f"n = {n}, d = {d}, nq = {nq}, metric = {metric}")
print(f"dataset size = {n * d * 4 / 1024 / 1024:.2f}MB")


n = 1000000, d = 128, nq = 10000, metric = L2
dataset size = 488.28MB


In [4]:
index = glass.Index(index_type="HNSW", dim=d, metric=metric, R=32, L=100)
g = index.build(X_train)


HNSW building progress: [10000/1000000]
HNSW building progress: [20000/1000000]
HNSW building progress: [30000/1000000]
HNSW building progress: [40000/1000000]
HNSW building progress: [50000/1000000]
HNSW building progress: [60000/1000000]
HNSW building progress: [70000/1000000]
HNSW building progress: [80000/1000000]
HNSW building progress: [90000/1000000]
HNSW building progress: [100000/1000000]
HNSW building progress: [110000/1000000]
HNSW building progress: [120000/1000000]
HNSW building progress: [130000/1000000]
HNSW building progress: [140000/1000000]
HNSW building progress: [150000/1000000]
HNSW building progress: [160000/1000000]
HNSW building progress: [170000/1000000]
HNSW building progress: [180000/1000000]
HNSW building progress: [190000/1000000]
HNSW building progress: [200000/1000000]
HNSW building progress: [210000/1000000]
HNSW building progress: [220000/1000000]
HNSW building progress: [230000/1000000]
HNSW building progress: [240000/1000000]
HNSW building progress: [

In [5]:
s = glass.Searcher(graph=g, data=X_train, metric=metric, level=2)
s.set_ef(36)


In [6]:
from os import cpu_count
from time import time
num_threads = cpu_count()

pred = s.batch_search(query=X_test, k=topk, num_threads=num_threads).reshape(-1, topk)
recall = dataset.evaluate(pred, topk)
print(f"Recall = {recall * 100:.2f}%")


Recall = 88.37%


In [7]:
s.batch_search(query=X_test, k=topk, num_threads=num_threads)  # warmup
for iter in range(10):
    t1 = time()
    pred = s.batch_search(query=X_test, k=topk, num_threads=num_threads)
    t2 = time()
    print(f"QPS = {nq / (t2 - t1):.2f}")


QPS = 197072.97
QPS = 198389.16
QPS = 198021.07
QPS = 199365.16
QPS = 196234.85
QPS = 199550.12
QPS = 195261.91
QPS = 164557.37
QPS = 195089.35
QPS = 200644.08


In [8]:
s.optimize()


=============Start optimization=============
settint best po = 10, best pl = 1
gaining 29.41% performance improvement
=============Done optimization=============


In [9]:
s.batch_search(query=X_test, k=topk, num_threads=num_threads)  # warmup
for iter in range(10):
    t1 = time()
    pred = s.batch_search(query=X_test, k=topk,
                          num_threads=num_threads).reshape(-1, topk)
    t2 = time()
    print(f"QPS = {nq / (t2 - t1)}")


QPS = 263056.47715513187
QPS = 263340.5536406045
QPS = 263789.3862970277
QPS = 262990.5006740446
QPS = 263401.7433243739
QPS = 260189.3276758353
QPS = 261101.22697476952
QPS = 260436.51311091657
QPS = 263138.9943222811
QPS = 242281.46282571903
